In [1]:
import tensorflow as tf

2023-05-29 20:03:50.255087: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-05-29 20:03:51.673748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-29 20:03:51.706579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-29 20:03:51.706730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [4]:
import sys
from pathlib import Path
import os
from datetime import datetime
import time
import random
import cv2 as cv
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.applications import EfficientNetB0
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Activation, Input, BatchNormalization, Multiply, Flatten
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, ReLU, Add, AveragePooling2D, MaxPooling2D, Input
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation


In [6]:
import numpy as np
import pandas as pd
import os

# Load the images
image_dir = '/home/vadim/Documents/ctci-python/yandex/yandex_43412_ml/dataset'
image_files = os.listdir(image_dir)


batch_size = 64
image_size = 64
input_shape = (image_size, image_size, 3)
dropout_rate = 0.4

# Load the pre-trained model
base_model = EfficientNetB0(weights="imagenet", 
                              include_top=False, 
                              input_shape=input_shape, 
                              drop_connect_rate=dropout_rate)

# Preprocess the images and extract features
image_features = {}
for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    features = base_model.predict(img_array)
    image_features[image_file] = features.flatten()

# Compute similarity matrix
similarity_matrix = np.zeros((len(image_files), len(image_files)))
for i, query_image in enumerate(image_files):
    query_features = image_features[query_image]
    for j, image in enumerate(image_files):
        if i != j:
            similarity_matrix[i][j] = np.dot(query_features, image_features[image])

# Find top 6 similar images for each query image
top_similar_images = {}
for i, query_image in enumerate(image_files):
    sorted_indices = np.argsort(similarity_matrix[i])[::-1][:6]
    similar_images = [image_files[idx] for idx in sorted_indices]
    top_similar_images[query_image] = similar_images

# Generate the submission CSV file
data = {'filename': [], 'ranking': []}
for query_image, similar_images in top_similar_images.items():
    data['filename'].append(query_image)
    data['ranking'].append(' '.join(similar_images))

df = pd.DataFrame(data)
df.to_csv('submission.csv', index=False)

2023-05-29 20:04:19.265348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


1/1 [==============================] - 0s 18ms/step


2023-05-29 20:04:21.041111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 17ms/step
